In [ ]:
import pandas as pd
import requests
import json
#import yaml
import time

from myTwitterLibrary import myTwitterLibrary as mtl

Target_File_Name='ChatGPT'

%load_ext autoreload
%autoreload 2

In [ ]:
#Suchfrage='(from:Nuklearia OR from:VeroWendland) lang:de'
#Suchfrage='(from:ausgestrahlt OR from:atomradar OR from:janhaverkamp OR from:XAhausX OR from:Urantransport) lang:de'
#Suchfrage="NanoBubbles OR @_Nano_Bubbles OR _Nano_Bubbles"
#Suchfrage='(from:BASE_bund OR from:strahlenschutz) lang:de'
#Suchfrage='(Atomkraft OR Atomenergie OR Atomreaktor OR Kernenergie OR Kernkraft OR Kernreaktor OR SMR) (Klima OR Klimawandel OR Klimaschutz)  lang:de'
#Suchfrage='(Endlager OR Endlagersuche OR Atommüll) (Fachkonferenz OR Teilgebiete OR Beteiligung OR beteiligen)  lang:de'
#SearchTerm="(#aiethics OR #sustainableAI OR #inclusiveai OR #greendata OR #responsibleai OR #trustworthyai OR #aiforgood OR #creativeai OR 3greenai OR #impactai OR #ai4prosperity OR #dataecosystems) -is:retweet"
#SearchTerm="(technologyassessment OR Technikfolgen OR (technology assessment)) -is:retweet"
#SearchTerm="Technikfolgen OR #TechnologyAssessment -is:retweet"
#SearchTerm="#BigData"
#SearchTerm='#GreenAI -is:retweet'
#SearchTerm="(#sustainableAI OR #inclusiveai OR #greendata OR #greenai)"
SearchTerm="(#AI (#sustainability OR #sustainable)) OR #SustainableAI OR #GreenAI" ### 2023-03-02
#SearchTerm="(GPT3, openAI OR ChatGPT OR ChatGPT3 OR GPT)"
#SearchTerm="(#GPT3, #openAI OR #ChatGPT OR #ChatGPT3 OR #GPT) (education OR uni OR university OR students OR school OR exam OR teaching OR teacher OR highschool) -is:retweet"


### Twitter Search



start_time='2010-01-01'
end_time='2023-03-30'


max_results=500 
query_params = {'query': SearchTerm,
                'start_time': mtl.dtC(start_time),
                'end_time': mtl.dtC(end_time),
                #'since_id': since_id,
                'max_results': 500,          # tweets per file --> important for large files
                'tweet.fields': 'author_id,created_at,entities,lang,public_metrics,in_reply_to_user_id,referenced_tweets,conversation_id,attachments',
                'user.fields': 'created_at,description,entities,id,location,name,username',
                'media.fields':'type,alt_text,url,height,preview_image_url',
                'expansions':'attachments.media_keys,referenced_tweets.id,referenced_tweets.id.author_id,entities.mentions.username,in_reply_to_user_id'
                }


#Target_File_Name="".join(filter(str.isalpha, SearchTerm.split(' ')[0]))
query_params

print(Target_File_Name, query_params)



startnumber=0


In [ ]:

for VersuchsNummer in range(5):
    try:
        for i in range(startnumber,1500): #100 bedeutet maximal 50.000 Tweets

            startnumber=i ## allows to restart  with the next number
            print(i,i*max_results)
            json_response=mtl.main(query_params)
            time.sleep(0.5)  
            NextToken=json_response['meta']['next_token']
            query_params["next_token"]=NextToken

            print(json_response["meta"])

            with open(mtl.Today+"_M_"+Target_File_Name+"__"+str(i)+'_data'+'.json', 'w', encoding='utf-8') as f:
                json.dump(json_response, f, ensure_ascii=False, indent=4)
    except:
        print("retry :D :D :D", VersuchsNummer,i)
        time.sleep(2)

In [ ]:
REAL_Next_Token=query_params["next_token"] ### Number 443
REAL_Next_Token

# Search with the Count engine

In [ ]:
start_time='2021-10-01'  ### Earliest: 2006-03-21T00:00Z
end_time='2023-04-07'
#SearchTerm='NanoBubbles OR @_Nano_Bubbles OR _Nano_Bubbles'


# Optional params: start_time,end_time,since_id,until_id,next_token,granularity
query_params = {'query': SearchTerm,
                'granularity': 'day', 
                'start_time': mtl.dtC(start_time),
                'end_time': mtl.dtC(end_time),
               }

print(query_params)
    

In [ ]:
ResultList=[]
for i in range(500): #100 bedeutet maximal 50.000 Tweets
    print(i)
    json_response=mtl.mainCount(query_params)
    time.sleep(0.05)
    Data=json_response["data"]
    
    NextToken=json_response['meta']['next_token']
    
    query_params["next_token"]=NextToken
    print(NextToken)
    ResultList.extend(Data)

In [ ]:
df=pd.DataFrame(ResultList)
df["Date"]=df["start"].apply(pd.to_datetime)
df.set_index("Date", inplace=True)

df["tweet_count"].to_csv(mtl.Today+"_"+Target_File_Name+"_Count_results.csv")

from textwrap import wrap

sampleSize="M"

PlotTitle="\n".join(wrap(f"Tweets per {sampleSize} on {SearchTerm} between {start_time} and {end_time}"))
sample=df.resample(sampleSize).sum()
ax=sample.plot(title=PlotTitle, figsize=(17,4))

import matplotlib.pyplot as plt

plt.title(PlotTitle, wrap=True)

plt.tight_layout(pad=1.08, h_pad=None, w_pad=None, rect=None)
plt.savefig(f"{mtl.Today}_{Target_File_Name}_2_History_tweets.pdf", dpi=300)


# Compare Counts

In [ ]:
df_Nanotechnology=pd.read_csv("22-06-14_nanotechnology_Count_results.csv", index_col="Date", converters={"Date":pd.to_datetime})
df_Nanotechnology.plot()

df_3D=pd.read_csv("22-06-14_3d printing_Count_results.csv", index_col="Date", converters={"Date":pd.to_datetime})
df_3D.plot()

In [ ]:
compare

In [ ]:
sampleSize="W"
CompTechs="Nano & 3D Printing"

compare=compare.join(df_3D, how='left', on="Date", lsuffix=' nanotechnology', rsuffix=' 3d-printing')

sample=compare.resample(sampleSize).sum()
ax=sample.plot(title=f"Tweets per {sampleSize} on {CompTechs} between {start_time} and {end_time}", figsize=(14,3))

In [ ]:
sampleSize="W"
CompTechs="Nanotechnology & 3D Printing & NanoParticles"

compare=compare.join(df["tweet_count"], how='left', on="Date", rsuffix=' Nanoparticle')

sample=compare.resample(sampleSize).sum()
ax=sample.plot(title=f"Tweets per {sampleSize} on {CompTechs} between {start_time} and {end_time}", figsize=(14,3))

In [ ]:
ax=sample.plot(title=f"Tweets per {sampleSize} on {CompTechs} between {start_time} and {end_time}", figsize=(14,3))

CompTechs="Nanotechnology-3D Printing-NanoParticles"

plt.tight_layout(pad=1.08, h_pad=None, w_pad=None, rect=None)
plt.savefig(f"{Heute}_{CompTechs}_History_tweets.jpg", dpi=300)


In [ ]:
sample.loc['2020-1-1':'2020-06-30'].plot()

### What happened March 2020 ? Covid?

# Combine Results / First look at data

In [ ]:
df=pd.DataFrame(json_response['data'])
df.head()

In [ ]:
### Combine Tweet Collections

combined_results = []

import glob
dataList=glob.glob("*"+DataName+"*"+"_data.json")
print(DataName)
with open(Heute+"_"+DataName+'combined_results.json', 'w', encoding='utf-8') as outfile:
    for file in dataList:
        f = open(file, 'r', encoding='utf-8')
        combined_results.append(json.load(f)["data"])
    json.dump(combined_results, outfile)
    


In [ ]:
df=pd.DataFrame()
data = json.load(open('22-02-09_NanoBiologycombined_results.json', encoding="utf8"))
for i in range(len(data)):
    dfx=pd.DataFrame(data[i])
    df=pd.concat(objs=[df,dfx])
    #df=df.append(dfx) 

In [ ]:
df=pd.DataFrame(json_response["data"])
df["entities"][1]

In [ ]:
def extractEntitites(entities):
    hashtagsListe, UrlListe, mentionListe,annotationsListe=[],[],[],[]
    #print(entities)
    if entities==entities:
        if "hashtags" in entities:
                hashtagsListe=[i["tag"] for i in entities["hashtags"] ]
        if 'urls' in entities:
            UrlListe=[i['expanded_url'] for i in entities['urls']]
        if "mentions" in entities:
            mentionListe=[i['username'] for i in entities["mentions"] ]
        if 'annotations' in entities:
            annotationsListe=[(i['type'],i['normalized_text']) for i in entities['annotations'] ]  
        res=(hashtagsListe, UrlListe, mentionListe, annotationsListe)
        return res

hashtagsListe, UrlListe, mentionListe,annotationsListe=zip(*df["entities"].apply(extractEntitites))
#df["hashtagsListe"], df["UrlListe"], df["mentionListe"]

In [ ]:
entities={"hashtagsListe":hashtagsListe, "UrlListe":UrlListe, 'mentionListe': mentionListe}


In [ ]:
hl=entities['hashtagsListe']
hashtags=[item for sublist in hl for item in sublist]
from collections import Counter
Counter(hashtags)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

flat_list = [item.title().strip() for sublist in hashtagsListe for item in sublist]
a_counter = Counter(flat_list)

most_common = a_counter.most_common(50)

wordcloud=WordCloud(background_color="white", width=1200, height=1200).generate_from_frequencies(dict(most_common))
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig(Heute+DataName+"_WordCloud.pdf",dpi=300)
plt.show()


In [ ]:
most_common = a_counter. most_common(15)[1:]
dhash=pd.DataFrame(most_common,columns=["Hashtag","counts"])

plt.figure()
dhash.set_index("Hashtag").plot.barh(figsize=(3,4),title="Co-Hashtags zu: "+Suchfrage)

plt.savefig(Heute+DataName+"_Hashtag-Barh.jpg",dpi=300,bbox_inches="tight")
plt.show()